### 1. Extração dos dados

O texto com os artigos da lei LGPD serão extraídos do site Planalto, em uma versão compilada.

In [ ]:
import requests
from bs4 import BeautifulSoup
import os

In [17]:
# Estabelecendo variáveis universais e header
URL = "https://www.planalto.gov.br/ccivil_03/_ato2015-2018/2018/lei/L13709compilado.htm"
RAW_DATA_PATH = os.path.join ("..", "data", "raw", "lgpd_raw.txt")

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36 Edg/138.0.0.0',
}

In [24]:
# Tentando obter resposta da URL
try:
    response = requests.get(URL, headers=headers)
    response.raise_for_status()
    print("Acesso bem sucedido!")
except Exception as e:
    print(f"Falha ao acessar a URL, erro: {e}")

Acesso bem sucedido!


In [25]:
# Estabelecendo o parser para html
soup = BeautifulSoup(response.content, 'html.parser')

# Encoding utilizado pelo site
response.encoding = 'windows-1252'

In [26]:
# Extraindo o texto alvo
lista_de_artigos = ""
try:
    lista_de_artigos = soup.select('p.Artigo, p.MsoNormal')
    
    if type(lista_de_artigos) != type(None):
        print(f"Extração bem sucedida")
    else:
        print("O texto não foi extraído e o resultado foi 'Nulo'")
        
except Exception as e:
    print(f"Erro na extração: {e}")


Extração bem sucedida


In [ ]:
# Por ser uma página mais antiga, é necessário alguns usos específicos de HTML
texto_bruto = []
if lista_de_artigos:
    print(f"Artigos encontrados. Preparando uma lista...\n")
    
    for i, artigo_tag in enumerate(lista_de_artigos):
        texto_do_artigo = artigo_tag.get_text(strip=True)
        texto_bruto.append(texto_do_artigo)
    print(f"Lista de artigos criada com sucesso: {len(lista_de_artigos)} artigos foram listados")
else:
    print("Nenhum parágrafo com a classe 'Artigo' foi encontrado.")

Artigos encontrados. Preparando uma lista...

Lista de artigos criada com sucesso: 396 artigos foram listados


In [28]:
texto_bruto[350:370]

['§ 4ş No cálculo do valor da multa de que trata o inciso II docaputdeste artigo, a autoridade nacional poderá considerar o faturamento total da empresa ou grupo de empresas, quando năo dispuser do valor do faturamento no ramo de atividade empresarial em que ocorreu a infraçăo, definido pela autoridade nacional, ou quando o valor for apresentado de forma incompleta ou năo for demonstrado de forma inequívoca e idônea.',
 'Art. 53. A autoridade nacional definirá, por meio de regulamento próprio sobre sançőes administrativas a infraçőes a esta Lei, que deverá ser objeto de consulta pública, as metodologias que orientarăo o cálculo do valor-base das sançőes de multa.(Vigęncia)',
 '§ 1ş As metodologias a que se refere ocaputdeste artigo devem ser previamente publicadas, para cięncia dos agentes de tratamento, e devem apresentar objetivamente as formas e dosimetrias para o cálculo do valor-base das sançőes de multa, que deverăo conter fundamentaçăo detalhada de todos os seus elementos, demon

In [31]:
# Convertendo lista para string para salvar com write
lgpd_raw = ' '.join(texto_bruto)

with open(RAW_DATA_PATH, 'w', encoding='utf-8') as f:
    f.write(lgpd_raw)

Essa primeira etapa está concluída. A reestruturação do código será feita para criar um script de extração do texto.

---

### 2. Processamento

Transformar o arquivo lgpd_raw.txt (único bloco de texto longo e com ruídos) em uma lista de "fragmentos de conhecimento" (chunks) limpos, estruturados e significativos.


In [ ]:
import spacy
import re

In [7]:
!python -q -m spacy download pt_core_news_lg

     ---------------------------------------- 0.0/568.2 MB ? eta -:--:--
     ---------------------------------------- 0.0/568.2 MB ? eta -:--:--
     ---------------------------------------- 0.3/568.2 MB ? eta -:--:--
     ---------------------------------------- 0.8/568.2 MB 2.2 MB/s eta 0:04:14
     ---------------------------------------- 2.4/568.2 MB 4.6 MB/s eta 0:02:03
     ---------------------------------------- 3.4/568.2 MB 4.7 MB/s eta 0:02:01
     ---------------------------------------- 6.0/568.2 MB 6.3 MB/s eta 0:01:30
      --------------------------------------- 7.6/568.2 MB 6.6 MB/s eta 0:01:25
      --------------------------------------- 9.4/568.2 MB 6.9 MB/s eta 0:01:21
      -------------------------------------- 11.3/568.2 MB 7.2 MB/s eta 0:01:18
      -------------------------------------- 13.1/568.2 MB 7.3 MB/s eta 0:01:16
     - ------------------------------------- 14.9/568.2 MB 7.5 MB/s eta 0:01:15
     - ------------------------------------- 16.8/568.2 MB 7.


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
# Carregando o modelo de português do Spacy (tamanho "large")
nlp = spacy.load("pt_core_news_lg", disable=["parser", "ner"])

In [18]:
# Carregnado arquivo
with open(RAW_DATA_PATH, 'r', encoding='utf-8') as f:
    raw_text = f.read()

In [ ]:
# Removendo as quebras de linhas múltiplas
texto_processado = re.sub(r'\n+', '\n', raw_text).strip()

# Otimizando o uso de espaços
texto_processado = re.sub(r' +', ' ', texto_processado)
print("Texto bruto limpo")

Texto bruto limpo


**Estratégia 1:** Quebrar o texto a cada 500 caracteres.

* Problema: Isso pode cortar uma frase no meio, destruindo o sentido.

**Estratégia 2:** Quebrar o texto por sentenças, usando o SpaCy.

* Problema: Para um texto jurídico, uma única sentença pode ser curta demais e não ter o contexto completo de um artigo.

**Estratégia 3:** Chunking Estrutural.

- Lógica: O próprio documento já nos dá a melhor estrutura: os Artigos (Art. 1º, Art. 2º, etc.) e Capítulos. Cada artigo é uma unidade de pensamento completa.

In [35]:
# --- CHUNKING ESTRUTURAL ---
# Estabelecer um padrão do inicio de cada artigo com . ou ° e capitulos com numero romano (estrutura da lei)
pattern = r'(Art\.\s\d+°?\.?|CAPÍTULO\s[IVXLCDM]+)'

# Dividir o texto com o pattern
parts = re.split(pattern, texto_processado)

Cada chunk é o delimitador como "Art. 1°" ou "Art. 23" mais o texto que o segue. Iremos alternar entre o texto e o delimitador e reagrupar. A estrutura será gravada em JSON como dicionário durante a execução do script.

In [36]:
chunks = []
for i in range(1, len(parts), 2):
    chunk_title = parts[i].strip()
    chunk_text = (parts[i] + parts[i+1]).strip()

    chunks.append({
        'id': len(chunks),
        'source': chunk_title,
        'text': chunk_text
    })

print(f"Texto dividido em {len(chunks)} chunks estruturados.")

Texto dividido em 76 chunks estruturados.


In [46]:
print(f"Uma pequena amostra dos chunks:\n\n{chunks[0]}\n{chunks[24]}\n{chunks[41]}")

Uma pequena amostra dos chunks:

{'id': 0, 'source': 'Art. 1', 'text': 'Art. 1ş Esta Lei dispőe sobre o tratamento de dados pessoais, inclusive nos meios digitais, por pessoa natural ou por pessoa jurídica de direito público ou privado, com o objetivo de proteger os direitos fundamentais de liberdade e de privacidade e o livre desenvolvimento da personalidade da pessoa natural. Parágrafo único. As normas gerais contidas nesta Lei săo de interesse \n\tnacional e devem ser observadas pela Uniăo, Estados, Distrito Federal e \n\tMunicípios.(Incluído pela Lei \n\tnş 13.853, de 2019)Vigęncia'}
{'id': 24, 'source': 'CAPÍTULO IVD', 'text': 'CAPÍTULO IVDO TRATAMENTO DE DADOS PESSOAIS PELO PODER PÚBLICO Seçăo IDas Regras'}
{'id': 41, 'source': 'Art. 37.', 'text': 'Art. 37. O controlador e o operador devem manter registro das operaçőes de tratamento de dados pessoais que realizarem, especialmente quando baseado no legítimo interesse.'}


A segunda etapa está concluída. A reestruturação do código será feita para criar um novo script de processamento do texto bruto e criação dos chunks.

---

### 3. Embeddings e Indexação

Converter os chunks de texto em vetores numéricos de alta dimensão e armazená-los em um banco de dados vetorial de alta velocidade (FAISS), ou seja, teremos o "cérebro" do assistente: uma representação matemática da LGPD que pode ser pesquisada por significado, não apenas por palavras-chave.

**O que são Embeddings de Texto?**

➥ Tecnicamente, é um vetor (uma lista) de números (ex: 384 ou 768 números) que captura a essência semântica de um texto.

**O que é um Banco de Dados Vetorial?**

➥ Um banco de dados vetorial é como um "bibliotecário" para vetores. Ele organiza os vetores de forma inteligente para encontrar outros vetores similares.

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

c:\Users\Thiag\OneDrive\Documentos\Projetos Python\lex-lgpd\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [53]:
# Modelo de embeddings
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

In [56]:
# Extrai os textos para criação dos embeddings
texts = [chunk['text'] for chunk in chunks]
print(f"Quantidade de chunks de texto para processar: {len(texts)}")

Quantidade de chunks de texto para processar: 76


In [55]:
embeddings = model.encode(texts, show_progress_bar=True)

Batches: 100%|██████████| 3/3 [00:05<00:00,  1.98s/it]


In [62]:
# O FAISS aceita apenas o formato float32, necessário uma conversão
embeddings = np.array(embeddings).astype('float32')

print(f"Embeddings gerados. Formato do vetor: {embeddings.shape}")

Embeddings gerados. Formato do vetor: (76, 384)


In [ ]:
# Dimensão
d = embeddings.shape[1] 

# Indice
index = faiss.IndexFlatL2(d)

# Atribuindo o índice aos embeddings
index.add(embeddings)

print(f"Índice FAISS criado. Total de vetores no índice: {index.ntotal}")

Índice FAISS criado. Total de vetores no índice: 76
